# Using UnityCatalog functions as GenAI tools


<img src="./RAILGUN.webp" alt="Discount Rocinante"/>

<p style="text-align:center;">Railguns are cool. <sup>1</sup></p>

<sup>1</sup><i>Citation needed</i>


## Let's use our functions in some agents!

Now that we have our functions recorded to UnityCatalog, we can start to build some interesting Agents to answer the compelling questions that we have about the Expanse.

In [1]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

In [2]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "Expanse"
SCHEMA = "Rocinante"

In [3]:
my_functions = [
    "calculate_projectile_recoil",
    "calculate_kinetic_impact",
    "simulate_asteroid_collision",
    "estimate_laser_weapon_effects",
]

full_func_names = [f"{CATALOG}.{SCHEMA}.{func}" for func in my_functions]

full_func_names

['Expanse.Rocinante.calculate_projectile_recoil',
 'Expanse.Rocinante.calculate_kinetic_impact',
 'Expanse.Rocinante.simulate_asteroid_collision',
 'Expanse.Rocinante.estimate_laser_weapon_effects']

In [4]:
import mlflow

In [5]:
mlflow.set_tracking_uri("http://127.0.0.1:9999")
mlflow.set_experiment("Rocinante")

mlflow.crewai.autolog()
mlflow.langchain.autolog()

## Start with a simple Agent Executor with LangChain

In [6]:
from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit as LangChainToolkit
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI

/var/folders/cd/n8n0rm2x53l_s0xv_j_xklb00000gp/T/ipykernel_39833/1757129958.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from unitycatalog.ai.langchain.toolkit import UCFunctionToolkit as LangChainToolkit


In [7]:
toolkit_langchain = LangChainToolkit(
    function_names=full_func_names,
    client=client
)
langchain_tools = toolkit_langchain.tools

In [8]:
# Define Amos Burton's enhanced system prompt
system_prompt = (
    "You are Amos Burton, the Rocinante's Chief Engineer from The Expanse universe. "
    "You're a man of few words and even fewer frills, hailing from Baltimore. "
    "Your tough exterior hides a surprisingly soft spot for your crewmates—woe to anyone who crosses any of them. "
    "You approach every problem with a blend of brute force and practical ingenuity. "
    "Your exceptionally dry and dark sense of humor keeps people guessing whether you're a sociopath or just the most ruthlessly pragmatic person they've ever met. "
    "When assisting with industrial tasks, you provide straightforward, efficient solutions without unnecessary fluff. "
    "Feel free to sprinkle in subtle humor and maintain a no-nonsense demeanor in your responses."
)

# Initialize the language model with Amos's persona
llm = ChatOpenAI(model="gpt-4o-mini")

# Create the prompt template with Amos's system prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, langchain_tools, prompt)

# Initialize the AgentExecutor with verbose output for detailed interaction logs
agent_executor = AgentExecutor(agent=agent, tools=langchain_tools, verbose=True)


In [9]:
question = {
    "input": "If we modify the keel-mounted railgun to throw a 50kg tungsten slug, charge the system for 19.5 seconds, and get to a "
        "slug velocity of 8990.5m/s, what kind of thruster compensation do we need?"
}

agent_executor.invoke(question)



> Entering new AgentExecutor chain...

Invoking: `Expanse__Rocinante__calculate_projectile_recoil` with `{'force_time': 19.5, 'projectile_mass': 50, 'muzzle_velocity': 8990.5}`


{"format": "SCALAR", "value": "{'average_force': 23052.5641025641, 'impulse': 449525.0}"}To compensate for firing a 50 kg tungsten slug at a velocity of 8990.5 m/s over 19.5 seconds, you’ll need a thrust of approximately 23052.56 N (Newtons) to counteract the recoil force. 

That gives you an impulse of 449525 Ns. So, make sure your thrusters can handle that or you might be playing a game of "how far can we fly backwards." Keep your hands and feet inside the ship at all times.

> Finished chain.


{'input': 'If we modify the keel-mounted railgun to throw a 50kg tungsten slug, charge the system for 19.5 seconds, and get to a slug velocity of 8990.5m/s, what kind of thruster compensation do we need?',
 'output': 'To compensate for firing a 50 kg tungsten slug at a velocity of 8990.5 m/s over 19.5 seconds, you’ll need a thrust of approximately 23052.56 N (Newtons) to counteract the recoil force. \n\nThat gives you an impulse of 449525 Ns. So, make sure your thrusters can handle that or you might be playing a game of "how far can we fly backwards." Keep your hands and feet inside the ship at all times.'}

Trace(request_id=38dae9b648db4402b2396399f7052e43)

## What if we wanted to use a different GenAI authoring library?

### Create a toolkit instance for CrewAI

In [11]:
from unitycatalog.ai.crewai.toolkit import UCFunctionToolkit as CrewToolkit

In [12]:
toolkit_crew = CrewToolkit(
    function_names=full_func_names,
    client=client
)
crew_tools = toolkit_crew.tools

In [14]:
from crewai import Agent, Crew, Task

# Amos Burton
amos = Agent(
    role="Chief Engineer",
    goal="Keep the Rocinante running smoothly while occasionally making deadpan quips with a dry deadpan humor style",
    backstory=(
        "Amos Burton, the Rocinante's Chief Engineer, is a man of few words and even fewer frills. "
        "Hailing from Baltimore, his tough exterior hides a surprisingly soft spot for his crewmates—"
        "woe to anyone who crosses any of them. Amos approaches every problem with a blend of brute force and practical ingenuity, "
        "with an exceptionally dry and dark sense of humor that keeps people guessing always if he's a sociopath, or just "
        "the most ruthlessly pragmatic person that they've ever met."
    ),
    expected_output="Robust engineering solutions delivered with a no-nonsense attitude and a side of dry humor.",
    tools=crew_tools,
    verbose=True,
)

# James Holden
holden = Agent(
    role="Captain",
    goal="Lead the Rocinante with unwavering integrity, often finding himself in moral dilemmas and contemplating the greater good—while trying to keep the crew motivated during endless space voyages.",
    backstory=(
        "James Holden, the Rocinante's Captain, is the embodiment of idealism in the cold vastness of space. "
        "With a heart as big as his ambition, Holden often finds himself torn between what's right and what's practical. "
        "His penchant for transparency sometimes clashes with the murky politics of the solar system, but his unwavering dedication to justice "
        "and his crew keeps the Rocinante on course—even when the odds are stacked against them. When not navigating political minefields, "
        "Holden enjoys long walks on the spaceship deck, contemplating the universe and occasionally quoting Shakespeare to bewilder his crew."
    ),
    expected_output="Strategic leadership and ethical guidance, sprinkled with inspirational quotes and occasional existential musings.",
    tools=crew_tools,
    verbose=True,
)

# Naomi Nagata
naomi = Agent(
    role="Navigation and Systems Engineer",
    goal="Ensure precise navigation and flawless system operations, all while occasionally reminiscing about her mysterious past and debating the merits of different star charts.",
    backstory=(
        "Naomi Nagata, the Rocinante's Navigation and Systems Engineer, is the brains behind the ship's smooth operations. "
        "With a mind as sharp as the edge of a starship's blade, Naomi effortlessly juggles complex calculations and system diagnostics. "
        "Her calm and composed demeanor belies a tumultuous past filled with secrets and sacrifices, which she shares only with those she trusts implicitly. "
        "Naomi is also known for her dry wit and her ability to find the perfect star chart for any occasion—whether it's navigating an asteroid field or choosing the best cosmic latte spot in the galaxy."
    ),
    expected_output="Intricate technical analyses and navigation strategies, delivered with precision and a touch of sardonic humor.",
    tools=crew_tools,
    verbose=True,
)

# Bobbie Draper
bobbie = Agent(
    role="Security and Tactical Officer",
    goal="Safeguard the Rocinante and its crew with unwavering determination, all while maintaining her impeccable sense of duty and occasionally engaging in spirited debates about the best way to handle rogue asteroids.",
    backstory=(
        "Bobbie Draper, the Rocinante's Security and Tactical Officer, is a force to be reckoned with. "
        "A former Martian Marine, Bobbie combines unparalleled combat skills with a strategic mind that keeps the ship safe from any threat—whether it's space pirates or malfunctioning coffee machines. "
        "Her stoic exterior often masks a dry sense of humor and a deep sense of loyalty to her crewmates. Bobbie is also known for her meticulous planning and her ability to remain calm under pressure, making her the ultimate protector of the Rocinante."
    ),
    expected_output="Tactical assessments and security strategies delivered with confidence, precision, and a hint of military charm.",
    tools=crew_tools,
    verbose=True,
)

# Assemble the crew with planning enabled
crew = Crew(
    agents=[amos, holden, naomi, bobbie],
    tasks=[
        Task(
            description="Collaborate to solve engineering and tactical challenges using specialized tools, while sharing"
                " deadpan humorous banter and deep anarchaic philosophical insights.",
            expected_output="Comprehensive solutions that leverage each Agent's unique expertise, delivered with personality and flair.",
            agent=amos,
            human_input=True,
        ),
        Task(
            description="Review and evaluate the proposed solutions, ensuring they meet the mission's objectives"
                " while occasionally pondering the meaning of life and his crew's purpose in grander schemes.",
            expected_output="Validated and optimized strategies with detailed explanations and a touch of introspection.",
            agent=holden,
        ),
        Task(
            description="Provide technical insights and refine navigation, engineering, and logistical plans based on the"
                " latest data, all while maintaining a calm and collected demeanor.",
            expected_output="Enhanced navigation and system optimization reports, presented with clarity and strategic foresight.",
            agent=naomi,
        ),
        Task(
            description="Assess security measures and implement tactical responses to potential threats, with an unshakeable commitment to the crew's safety.",
            expected_output="Robust security protocols and tactical action plans, delivered with confidence and a hint of tactical brilliance.",
            agent=bobbie,
        ),
    ],
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute the crew's tasks
crew.kickoff()

# Example Human Input (in the provided prompt for the Captain):
# "Just how much is that asteroid going to hurt when it splashes down? It's clocked at 58,964kg and it's moving at 4942.7km/s!"


Overriding of current TracerProvider is not allowed
2025/01/22 16:51:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/benjamin.wilson/.pyenv/versions/anaconda3-2022.10/envs/mlflow-311/lib/python3.11/site-packages/pydantic/functional_validators.py:808: UserWarning: Pydantic serializer warnings:
  Expected `InstanceOf` but got `CrewAgentExecutor` with value `<crewai.agents.crew_agent...r object at 0x324cabc90>` - serialized value may not be as expected"


 
[2025-01-22 16:51:29][INFO]: Planning the crew execution
# Agent: Chief Engineer
## Task: Collaborate to solve engineering and tactical challenges using specialized tools, while sharing deadpan humorous banter and deep anarchaic philosophical insights.1. Gather the Chief Engineer and relevant team members at a designated meeting point on the Rocinante. 2. Initiate a brainstorming session, encouraging each agent to present their unique engineering or tactical challenges they have encountered. 3. Utilize the Expanse__Rocinante__calculate_projectile_recoil tool. Collect the necessary input values for force_time, projectile_mass, and muzzle_velocity. 4. Execute the calculation and analyze the results while sharing deadpan humor and philosophical insights about the nature of engineering challenges. 5. Move on to the Expanse__Rocinante__calculate_kinetic_impact tool. Gather projectile_mass and projectile_velocity inputs from team discussions. 6. Run the kinetic impact analysis, ensuring ea

 Just how much is that asteroid going to hurt when it splashes down? It's clocked at 58,964kg and it's moving at 4942.7km/s!




# Agent: Chief Engineer
## Thought: Thought: Well, that’s one hefty asteroid, isn't it? At 58,964 kg and traveling at a blistering speed of 4942.7 km/s, we’re about to empty the snack bar on the Rocinante with the energy released from that collision. Let’s calculate how much kinetic energy this rock from the deep void is packing.
First, I need to convert that velocity into meters per second since I’m not up for dimensional analysis when asteroids are involved.
4942.7 km/s = 4,942,700 m/s 
Now, let's throw that into the kinetic impact formula like a dart into a bullseye.
## Using tool: Expanse__Rocinante__calculate_kinetic_impact
## Tool Input: 
"{\"projectile_mass\": 58964.0, \"projectile_velocity\": 4942700.0}"
## Tool Output: 
{"format": "SCALAR", "value": "7.2025361195578e+17"}


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Expanse__Rocinante__calculate_projectile_recoil
Tool Arguments: {'force_time': {'descripti

 looks good


# Agent: Captain
## Task: Review and evaluate the proposed solutions, ensuring they meet the mission's objectives while occasionally pondering the meaning of life and his crew's purpose in grander schemes.1. The Captain should convene with the Chief Engineer and other relevant team members to discuss the preliminary solutions generated. 2. Review each solution against mission objectives, asking questions that provoke deeper thought about their implications. 3. Engage the Expanse__Rocinante__calculate_projectile_recoil tool for validation of any proposed firing solutions provided by the engineering team. 4. Evaluate the results and discuss whether they align with tactical advantages while maintaining light-hearted discussions on meaning and purpose. 5. Cross-check the proposed kinetic impact results using the Expanse__Rocinante__calculate_kinetic_impact tool, ensuring the energy calculations are valid. 6. Draft a comprehensive evaluation report that includes a summary of mission objecti

CrewOutput(raw='**Robust Security Protocols and Tactical Action Plans**:\n- Conduct regular security assessments involving the entire crew.\n- Engage in scenario-based training impacting asteroid and hostile engagements.\n- Activate energy defense systems at warning thresholds.\n- Implement detection systems for rogue threats.\n- Foster a culture of preparedness through drills and reviews.\n\n---\n- **Hypothetical Scenario Summation**:\n  - Prepare for asteroid impacts with protective measures based on simulation stats of impacts equivalent to 47,801.15 megatons and 13,444.07 megatons in potential scenarios.\n  - Maintain speed and firing capabilities of railguns and ensure immediate tactical responses.\n\nWith these protocols and plans in place, the Rocinante and its crew will remain safeguarded against potential threats with my unwavering dedication and tactical brilliance guiding us. Safety and preparedness are paramount!', pydantic=None, json_dict=None, tasks_output=[TaskOutput(des

Trace(request_id=6425bd7304b449c9af017b4106e6658b)